In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

import tensorflow as tf
import random
import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


labels.csv
sample_submission.csv
test
train



In [2]:
##############
# parameters #
##############
# dontFreezeLast = 0;

# patience = 10;

# gpuName = '/device:GPU:0'
# workers = 2;
# histogram_freq = 0;

# epochs = 100;
# validation_size=0.3;

modelPath = '../models/VGG16_opt/';
modelName = 'run3.h5';
os.environ["CUDA_VISIBLE_DEVICES"]="1"

if not os.path.exists(modelPath):
    os.makedirs(modelPath)

First we will read in the csv's so we can see some more information on the filenames and breeds

In [3]:
# df_train = pd.read_csv('../input/labels.csv')
# df_test = pd.read_csv('../input/sample_submission.csv')

# print('Training images: ',df_train.shape[0])
# print('Test images: ',df_test.shape[0])

# reduce dimensionality
#df_train = df_train.head(100)
#df_test = df_test.head(100)

In [4]:
#df_train.head(10)

We can see that the breed needs to be one-hot encoded for the final submission, so we will now do this.

In [5]:
# targets_series = pd.Series(df_train['breed'])
# one_hot = pd.get_dummies(targets_series, sparse = True)

In [6]:
# one_hot_labels = np.asarray(one_hot)

Next we will read in all of the images for test and train, using a for loop through the values of the csv files. I have also set an im_size variable which sets the size for the image to be re-sized to, 90x90 px, you should play with this number to see how it affects accuracy.

In [7]:
# im_size = 90

In [8]:
# x_train = []
# y_train = []
# x_test = []

In [9]:
# i = 0 
# for f, breed in tqdm(df_train.values[:10]):
#     img = cv2.imread('../input/train/{}.jpg'.format(f))
#     label = one_hot_labels[i]
#     x_train.append(cv2.resize(img, (im_size, im_size)))
#     y_train.append(label)
#     i += 1

In [10]:
# for f in tqdm(df_test['id'].values):
#     img = cv2.imread('../input/test/{}.jpg'.format(f))
#     x_test.append(cv2.resize(img, (im_size, im_size)))

In [11]:
# y_train_raw = np.array(y_train, np.uint8)
# x_train_raw = np.array(x_train, np.float32) / 255.
# x_test  = np.array(x_test, np.float32) / 255.

We check the shape of the outputs to make sure everyting went as expected.

In [12]:
# print(x_train_raw.shape)
# print(y_train_raw.shape)
# print(x_test.shape)

We can see above that there are 120 different breeds. We can put this in a num_class variable below that can then be used when creating the CNN model.

In [13]:
# num_class = y_train_raw.shape[1]
# print('Number of classes: ', num_class)

It is important to create a validation set so that you can gauge the performance of your model on independent data, unseen to the model in training. We do this by splitting the current training set (x_train_raw) and the corresponding labels (y_train_raw) so that we set aside 30 % of the data at random and put these in validation sets (X_valid and Y_valid).

* This split needs to be improved so that it contains images from every class, with 120 separate classes some can not be represented and so the validation score is not informative.

In [14]:
# X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=validation_size, random_state=1)

Now we build the CNN architecture. Here we are using a pre-trained model VGG19 which has already been trained to identify many different dog breeds (as well as a lot of other objects from the imagenet dataset see here for more information: http://image-net.org/about-overview). Unfortunately it doesn't seem possible to downlod the weights from within this kernel so make sure you set the weights argument to 'imagenet' and not None, as it currently is below.

We then remove the final layer and instead replace it with a single dense layer with the number of nodes corresponding to the number of breed classes we have (120).

In [15]:
def data():
    print('Getting data')
    df_train = pd.read_csv('../input/labels.csv')
    df_test = pd.read_csv('../input/sample_submission.csv')
    
    targets_series = pd.Series(df_train['breed'])
    one_hot = pd.get_dummies(targets_series, sparse = True)
    one_hot_labels = np.asarray(one_hot)
    
    im_size = 90;
    x_train = []
    y_train = []
    x_test = []
    
    i = 0 
    for f, breed in tqdm(df_train.values):
        img = cv2.imread('../input/train/{}.jpg'.format(f))
        label = one_hot_labels[i]
        x_train.append(cv2.resize(img, (im_size, im_size)))
        y_train.append(label)
        i += 1
    
    y_train_raw = np.array(y_train, np.uint8)
    x_train_raw = np.array(x_train, np.float32) / 255.
    num_class = y_train_raw.shape[1]
    
    print('Splitting into training/validation')
    X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.3, random_state=1)
    
    return X_train, Y_train, X_valid, Y_valid

In [16]:
# prepare data and model for hyperas

def model(X_train,Y_train,X_valid,Y_valid):
    print('Creating model')
    base_model = VGG16(weights = 'imagenet',
                       include_top=False,
                       input_shape=(im_size, im_size, 3))

    dropout=0.6;
    dontFreeze = 3;
    batchSize = 64;
    momentum=0.99;
    #
    lambda_l2 = {{uniform(0.000001,0.1)}};
    bottleneckFlag = {{choice([True,False])}};
    
    print()
    print('lambda_l2=',lambda_l2)
    print('bottleneckFlag=',bottleneckFlag)
    print()
    
    stepsPerEpoch = round( len(X_train)/batchSize );
    
    # Add a new top layer
    x = base_model.output
    x = BatchNormalization(axis=1,momentum=momentum)(x)
    x = Flatten()(x)
    x = Dropout(dropout)(x)
    
    x = Dense(2048,activation='relu',kernel_regularizer=l2(lambda_l2))(x)
    if bottleneckFlag:
        x = Dense(100,activation='relu')(x)
    x = Dense(1024,activation='relu')(x)
    x = Dense(512,activation='relu')(x)
    # in any case:
    predictions = Dense(num_class, activation='softmax')(x)

    # This is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    # First: train only the top layers (which were randomly initialized)
    for i in range(len(base_model.layers)-dontFreeze):
        base_model.layers[i].trainable = False

    # predetermined optimizer
    lr=0.00020389590556056983;
    beta_1=0.9453158868247398;
    beta_2=0.9925872692991417;
    decay=0.000821336141287018;
    adam = keras.optimizers.Adam(lr=lr,beta_1=beta_1,beta_2=beta_2,decay=decay)
    model.compile(loss='categorical_crossentropy', 
                  optimizer=adam, 
                  metrics=['accuracy'])

    callbacks_list = [];
    callbacks_list.append(keras.callbacks.EarlyStopping(
        monitor='val_acc',
        patience=10,
        verbose=1));


    # data augmentation & fitting
    datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.5,
        zoom_range=0.5,
        horizontal_flip=True,
        vertical_flip=True);
    
    model.fit_generator(
        datagen.flow(X_train,Y_train,batch_size=batchSize),
        steps_per_epoch=stepsPerEpoch,
        epochs=150,
        verbose=1,
        validation_data=(X_valid,Y_valid),
        workers=2,
        shuffle=True,
        callbacks=callbacks_list)
#     model.fit(X_train, Y_train,
#       epochs=100,
#       batch_size = batchSize,
#       validation_data=(X_valid, Y_valid),
#       verbose=1,
#       callbacks=callbacks_list)

    score, acc = model.evaluate(X_valid, Y_valid, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}


In [17]:
best_run, best_model = optim.minimize(model=model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='VGG16Optimization_3')

>>> Imports:
#coding=utf-8

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from keras.applications import VGG16
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.layers import Dense, Dropout, Flatten, BatchNormalization
except:
    pass

try:
    from keras.regularizers import l2
except:
    pass

try:
    from keras.preprocessing.image import ImageDataGenerator
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    import random
except:
    pass

try:
    import os
except:
    pass

try:
    from tqdm import tqdm
except:
    pass

try:
    from sklearn import preprocessing
except:
    pass

try:
    from sklearn.model_selectio

100%|██████████| 10222/10222 [00:59<00:00, 173.05it/s]


Splitting into training/validation
Creating model

lambda_l2= 0.04371218882692479
bottleneckFlag= True

Epoch 1/150
112/112 [==============================] - 36s 320ms/step - loss: 45.5597 - acc: 0.0088 - val_loss: 16.4873 - val_acc: 0.0095
Epoch 2/150
112/112 [==============================] - 28s 254ms/step - loss: 9.1804 - acc: 0.0102 - val_loss: 5.6495 - val_acc: 0.0098
Epoch 3/150
112/112 [==============================] - 27s 242ms/step - loss: 5.0736 - acc: 0.0115 - val_loss: 4.8472 - val_acc: 0.0098
Epoch 4/150
112/112 [==============================] - 26s 231ms/step - loss: 4.8119 - acc: 0.0121 - val_loss: 4.7795 - val_acc: 0.0153
Epoch 5/150
112/112 [==============================] - 26s 234ms/step - loss: 4.7130 - acc: 0.0206 - val_loss: 4.6292 - val_acc: 0.0235
Epoch 6/150
112/112 [==============================] - 28s 246ms/step - loss: 4.6083 - acc: 0.0269 - val_loss: 4.5212 - val_acc: 0.0271
Epoch 7/150
112/112 [==============================] - 28s 252ms/step - loss: 

112/112 [==============================] - 29s 255ms/step - loss: 3.4969 - acc: 0.1660 - val_loss: 3.5138 - val_acc: 0.1692
Epoch 61/150
112/112 [==============================] - 28s 247ms/step - loss: 3.4922 - acc: 0.1685 - val_loss: 3.5112 - val_acc: 0.1777
Epoch 62/150
112/112 [==============================] - 27s 240ms/step - loss: 3.5076 - acc: 0.1697 - val_loss: 3.5106 - val_acc: 0.1793
Epoch 63/150
112/112 [==============================] - 25s 226ms/step - loss: 3.5004 - acc: 0.1643 - val_loss: 3.5187 - val_acc: 0.1705
Epoch 64/150
112/112 [==============================] - 27s 239ms/step - loss: 3.4696 - acc: 0.1715 - val_loss: 3.5322 - val_acc: 0.1722
Epoch 65/150
112/112 [==============================] - 28s 250ms/step - loss: 3.4577 - acc: 0.1751 - val_loss: 3.5098 - val_acc: 0.1718
Epoch 66/150
112/112 [==============================] - 29s 258ms/step - loss: 3.4773 - acc: 0.1695 - val_loss: 3.5245 - val_acc: 0.1787
Epoch 67/150
112/112 [==============================] 

Epoch 120/150
112/112 [==============================] - 29s 263ms/step - loss: 3.1674 - acc: 0.2318 - val_loss: 3.4094 - val_acc: 0.2031
Epoch 121/150
112/112 [==============================] - 29s 257ms/step - loss: 3.1731 - acc: 0.2231 - val_loss: 3.3559 - val_acc: 0.2155
Epoch 122/150
112/112 [==============================] - 27s 244ms/step - loss: 3.1783 - acc: 0.2262 - val_loss: 3.3844 - val_acc: 0.2113
Epoch 123/150
112/112 [==============================] - 26s 230ms/step - loss: 3.1524 - acc: 0.2253 - val_loss: 3.3625 - val_acc: 0.2145
Epoch 124/150
112/112 [==============================] - 26s 235ms/step - loss: 3.1646 - acc: 0.2338 - val_loss: 3.3680 - val_acc: 0.2152
Epoch 00124: early stopping
Test accuracy: 0.215194000657
Creating model

lambda_l2= 0.038381502234122906
bottleneckFlag= False

Epoch 1/150
112/112 [==============================] - 31s 280ms/step - loss: 44.1257 - acc: 0.0098 - val_loss: 18.6486 - val_acc: 0.0072
Epoch 2/150
112/112 [======================

...

Epoch 51/150
112/112 [==============================] - 25s 227ms/step - loss: 3.1608 - acc: 0.2696 - val_loss: 3.4905 - val_acc: 0.2172
Epoch 52/150
112/112 [==============================] - 26s 229ms/step - loss: 3.0925 - acc: 0.2737 - val_loss: 3.5001 - val_acc: 0.2185
Epoch 53/150
112/112 [==============================] - 25s 227ms/step - loss: 3.1068 - acc: 0.2711 - val_loss: 3.4566 - val_acc: 0.2227
Epoch 54/150
112/112 [==============================] - 25s 226ms/step - loss: 3.0999 - acc: 0.2689 - val_loss: 3.4817 - val_acc: 0.2233
Epoch 55/150
112/112 [==============================] - 25s 225ms/step - loss: 3.0691 - acc: 0.2838 - val_loss: 3.4963 - val_acc: 0.2276
Epoch 56/150
112/112 [==============================] - 25s 225ms/step - loss: 3.0500 - acc: 0.2822 - val_loss: 3.4639 - val_acc: 0.2273
Epoch 57/150
112/112 [==============================] - 25s 226ms/step - loss: 3.0507 - acc: 0.2892 - val_loss: 3.4713 - val_acc: 0.2227
Epoch 58/150
112/112 [===================

Epoch 20/150
112/112 [==============================] - 25s 226ms/step - loss: 3.7496 - acc: 0.1478 - val_loss: 3.6638 - val_acc: 0.1637
Epoch 21/150
112/112 [==============================] - 25s 225ms/step - loss: 3.7479 - acc: 0.1487 - val_loss: 3.6890 - val_acc: 0.1673
Epoch 22/150
112/112 [==============================] - 25s 226ms/step - loss: 3.7077 - acc: 0.1560 - val_loss: 3.6086 - val_acc: 0.1679
Epoch 23/150
112/112 [==============================] - 25s 226ms/step - loss: 3.6920 - acc: 0.1607 - val_loss: 3.6130 - val_acc: 0.1784
Epoch 24/150
112/112 [==============================] - 25s 226ms/step - loss: 3.6822 - acc: 0.1606 - val_loss: 3.5998 - val_acc: 0.1780
Epoch 25/150
112/112 [==============================] - 25s 226ms/step - loss: 3.6389 - acc: 0.1648 - val_loss: 3.6324 - val_acc: 0.1738
Epoch 26/150
112/112 [==============================] - 25s 226ms/step - loss: 3.6036 - acc: 0.1761 - val_loss: 3.5949 - val_acc: 0.1819
Epoch 27/150
112/112 [===================

Epoch 80/150
112/112 [==============================] - 25s 226ms/step - loss: 2.9895 - acc: 0.2876 - val_loss: 3.3026 - val_acc: 0.2491
Epoch 81/150
112/112 [==============================] - 25s 226ms/step - loss: 2.9586 - acc: 0.2950 - val_loss: 3.2979 - val_acc: 0.2468
Epoch 82/150
112/112 [==============================] - 25s 226ms/step - loss: 2.9895 - acc: 0.2836 - val_loss: 3.3148 - val_acc: 0.2426
Epoch 83/150
112/112 [==============================] - 25s 226ms/step - loss: 2.9448 - acc: 0.2962 - val_loss: 3.3094 - val_acc: 0.2416
Epoch 84/150
112/112 [==============================] - 25s 226ms/step - loss: 2.9581 - acc: 0.2891 - val_loss: 3.3455 - val_acc: 0.2370
Epoch 85/150
112/112 [==============================] - 25s 226ms/step - loss: 2.9183 - acc: 0.3014 - val_loss: 3.3246 - val_acc: 0.2426
Epoch 86/150
112/112 [==============================] - 25s 226ms/step - loss: 2.9207 - acc: 0.3007 - val_loss: 3.3157 - val_acc: 0.2478
Epoch 87/150
112/112 [===================

Epoch 22/150
112/112 [==============================] - 25s 226ms/step - loss: 3.7851 - acc: 0.1382 - val_loss: 3.6822 - val_acc: 0.1588
Epoch 23/150
112/112 [==============================] - 25s 226ms/step - loss: 3.7615 - acc: 0.1392 - val_loss: 3.6523 - val_acc: 0.1640
Epoch 24/150
112/112 [==============================] - 25s 226ms/step - loss: 3.7591 - acc: 0.1397 - val_loss: 3.6626 - val_acc: 0.1620
Epoch 25/150
112/112 [==============================] - 25s 226ms/step - loss: 3.7344 - acc: 0.1493 - val_loss: 3.6260 - val_acc: 0.1718
Epoch 26/150
112/112 [==============================] - 25s 226ms/step - loss: 3.7217 - acc: 0.1439 - val_loss: 3.6790 - val_acc: 0.1549
Epoch 27/150
112/112 [==============================] - 25s 226ms/step - loss: 3.6882 - acc: 0.1547 - val_loss: 3.6143 - val_acc: 0.1647
Epoch 28/150
112/112 [==============================] - 25s 225ms/step - loss: 3.6866 - acc: 0.1515 - val_loss: 3.5714 - val_acc: 0.1777
Epoch 29/150
112/112 [===================

Epoch 82/150
112/112 [==============================] - 25s 226ms/step - loss: 3.1191 - acc: 0.2580 - val_loss: 3.3553 - val_acc: 0.2266
Epoch 83/150
112/112 [==============================] - 26s 234ms/step - loss: 3.0816 - acc: 0.2584 - val_loss: 3.3391 - val_acc: 0.2233
Epoch 84/150
112/112 [==============================] - 25s 225ms/step - loss: 3.1175 - acc: 0.2547 - val_loss: 3.3237 - val_acc: 0.2237
Epoch 85/150
112/112 [==============================] - 25s 226ms/step - loss: 3.0912 - acc: 0.2576 - val_loss: 3.3294 - val_acc: 0.2302
Epoch 86/150
112/112 [==============================] - 25s 226ms/step - loss: 3.0640 - acc: 0.2699 - val_loss: 3.3657 - val_acc: 0.2253
Epoch 87/150
112/112 [==============================] - 25s 225ms/step - loss: 3.0665 - acc: 0.2646 - val_loss: 3.3278 - val_acc: 0.2246
Epoch 88/150
112/112 [==============================] - 25s 225ms/step - loss: 3.0876 - acc: 0.2564 - val_loss: 3.3337 - val_acc: 0.2240
Epoch 89/150
112/112 [===================

In [ ]:
X_train, Y_train, X_test, Y_test = data()
val_loss, val_acc = best_model.evaluate(X_test, Y_test);
print("Evalutation of best performing model:")
print("Validation loss: ", val_loss)
print("Validation accuracy: ", val_acc)

Getting data


 48%|████▊     | 4908/10222 [00:25<00:27, 194.64it/s]

In [ ]:
best_model.save(modelPath+modelName);
best_model.summary()

Remember, accuracy is low here because we are not taking advantage of the pre-trained weights as they cannot be downloaded in the kernel. This means we are training the wights from scratch and I we have only run 1 epoch due to the hardware constraints in the kernel.

Next we will make our predictions.

In [ ]:
# preds = model.predict(x_test, verbose=1)

In [ ]:
# sub = pd.DataFrame(preds)
# # Set column names to those generated by the one-hot encoding earlier
# col_names = one_hot.columns.values
# sub.columns = col_names
# # Insert the column id from the sample_submission at the start of the data frame
# sub.insert(0, 'id', df_test['id'])
# sub.head(10)